# Retrieval Augmented Generation Demo - Inventory (partial)

## Prerequisites

### Make a `.env` file

```sh
cp .env.example .env
```

### Provide API keys in `.env` file 

- LANGCHAIN_API_KEY (optional)
  -  Sign up to https://smith.langchain.com/
  -  Go to Settings -> API Keys
  -  Create API Key
- GROQ_API_KEY
  - Sign up to https://console.groq.com/
  - Go to API Keys
  - Create API Key

### Check that everything is working

- Check that the configured Kernel is `.venv`
  - In the navigation bar, click `Kernel`
  - Click `Change Kernel...`
- Check that all cells are working
  - In the navigation bar, click `Kernel`
  - Click `Restart Kernel...`
  - In the navigation bar, click `Edit`
  - Click `Clear Outputs of All Cells`
  - In the navigation bar, click `Run`
  - Click `Run All Cells`

## Set up environment

In [33]:
import warnings
from dotenv import load_dotenv

warnings.filterwarnings('ignore')
load_dotenv()

True

## Set up tracing (Optional)
Define `LANGCHAIN_API_KEY` in your `.env` file.  

In [34]:
import os

if "LANGCHAIN_API_KEY" in os.environ and os.environ["LANGCHAIN_API_KEY"] != "":
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
else:
    os.environ["LANGCHAIN_TRACING_V2"] = "false"

os.environ["LANGCHAIN_TRACING_V2"]

'true'

## Set up LLM
Define `GROQ_API_KEY` in your `.env` file.

In [35]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

## Indexing - Load documents

In [36]:
import os
import bs4
from langchain_community.document_loaders import WebBaseLoader

os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"

# bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=(
        "https://k-series-support.lightspeedhq.com/hc/en-us/articles/4407517428891-About-Inventory",
        "https://k-series-support.lightspeedhq.com/hc/en-us/articles/20537741099419-Discrepancy-reports",
        "https://k-series-support.lightspeedhq.com/hc/en-us/articles/16092679490331-Stock-transfers",
        "https://k-series-support.lightspeedhq.com/hc/en-us/articles/6752297865883-About-Purchase",
        "https://k-series-support.lightspeedhq.com/hc/en-us/articles/4407560855707-Adding-suppliers",
        "https://k-series-support.lightspeedhq.com/hc/en-us/articles/4407569259547-Creating-purchase-orders",
        "https://k-series-support.lightspeedhq.com/hc/en-us/articles/4407576020507-Receiving-managing-and-deleting-purchase-orders",
        "https://k-series-support.lightspeedhq.com/hc/en-us/articles/6734173537435-Setting-up-recurring-orders",
        "https://k-series-support.lightspeedhq.com/hc/en-us/articles/11207355170203-Purchase-Reports",
        "https://k-series-support.lightspeedhq.com/hc/en-us/articles/4407511020571-About-Produce",
        "https://k-series-support.lightspeedhq.com/hc/en-us/articles/4407511552155-Creating-and-managing-recipes",
        "https://k-series-support.lightspeedhq.com/hc/en-us/articles/4407567874843-Creating-and-managing-batches",
        "https://k-series-support.lightspeedhq.com/hc/en-us/articles/4407568240795-Batch-history",
        # "https://k-series-support.lightspeedhq.com/hc/en-us/articles/4407509542043-Stock-management-Inventory",
        # "https://k-series-support.lightspeedhq.com/hc/en-us/articles/6669636140443-About-Items-Inventory",
        # "https://k-series-support.lightspeedhq.com/hc/en-us/articles/4407517612699-Stock-levels",
        # "https://k-series-support.lightspeedhq.com/hc/en-us/articles/6670276492699-Stock-locations",
        # "https://k-series-support.lightspeedhq.com/hc/en-us/articles/4407510354715-Performing-stock-counts",
        # "https://k-series-support.lightspeedhq.com/hc/en-us/articles/6686397618843-Par-levels",
        # "https://k-series-support.lightspeedhq.com/hc/en-us/articles/7116014889499-Wastage",
    ),
    # bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

len(docs[0].page_content)

3999

## Indexing - Split documents

In [38]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True,
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

104

## Indexing - Embed and store documents

In [39]:
import time
from langchain_chroma import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

print("Indexing started...")
start = time.time()

vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=HuggingFaceEmbeddings()
)

end = time.time()
print(f"Indexing completed in {round(end - start, 2)}s.")

Indexing started...
Indexing completed in 6.09s.


## Retrieval and generation

In [41]:
import os
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 6},
)
prompt = hub.pull("rlm/rag-prompt")
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

def ask(question):
  for chunk in rag_chain.stream(question):
    print(chunk, end="", flush=True)  

### What is Inventory? What are its features?

In [42]:
ask("What is Inventory? What are its features?")

Inventory refers to the management and tracking of goods or stock levels in a business. Its features include stock management, produce management, and purchase management, allowing for the tracking of quantity and value of items, setting of reorder points, and creation of purchase orders.

## Is there a feature I can use to define instructions for preparing dishes and listing the ingredients required?

In [44]:
ask("Is there a feature I can use to define instructions for preparing dishes and listing the ingredients required?")

Yes, you can define instructions for preparing dishes and list the ingredients required by setting up a new recipe in the Back Office > Menu Management > Items. Ensure that the finished product and its ingredients exist as items before creating a new recipe.

## Is there a feature I can use to record item wastages?

In [45]:
ask("Is there a feature I can use to record item wastages?")

No, there is no feature to record item wastages in the given context.